In [ ]:
import numpy as np
import meshio
import trimesh
import ray
import pyvista as pv

# Initialize Ray
ray.init(ignore_reinit_error=True)

# Load the STL file using meshio
mesh = meshio.read('/home/dave/Downloads/surface_cut.stl')

# Extract vertices and faces
vertices = mesh.points
faces = mesh.cells_dict['triangle']

# Define the bounds of the grid
x_min, x_max = vertices[:, 0].min(), vertices[:, 0].max()
y_min, y_max = vertices[:, 1].min(), vertices[:, 1].max()
z_min, z_max = vertices[:, 2].min(), vertices[:, 2].max()

# Define the resolution of the grid
resolution = 50  # Reduce for faster testing

# Create the grid
x = np.linspace(x_min, x_max, resolution)
y = np.linspace(y_min, y_max, resolution)
z = np.linspace(z_min, z_max, resolution)
xx, yy, zz = np.meshgrid(x, y, z, indexing='ij')
points = np.vstack([xx.ravel(), yy.ravel(), zz.ravel()]).T

# Convert the mesh data to a trimesh object
tri_mesh = trimesh.Trimesh(vertices=vertices, faces=faces)

# Function to check points in chunks
@ray.remote
def check_points_chunk(chunk):
    return tri_mesh.contains(chunk)

# Split points into chunks for parallel processing
num_chunks = 8
chunks = np.array_split(points, num_chunks)

# Run the point-in-mesh check in parallel using Ray
futures = [check_points_chunk.remote(chunk) for chunk in chunks]
inside_chunks = ray.get(futures)

# Combine the results
inside = np.concatenate(inside_chunks)

# Reshape the inside array to the grid shape
mask = inside.reshape((resolution, resolution, resolution))

# Extract the coordinates of the points inside the mask
indices = np.argwhere(mask)
inside_points = points[mask.ravel()]

# Save the points to different file formats

# CSV File
np.savetxt('inside_points.csv', inside_points, delimiter=',', header='x,y,z', comments='')

# TXT File
np.savetxt('inside_points.txt', inside_points, fmt='%.6f', header='x y z', comments='')

# Binary NumPy File
np.save('inside_points.npy', inside_points)

# PyVista PLY File
point_cloud = pv.PolyData(inside_points)
point_cloud.save('inside_points.ply')

# Optional: Visualize with PyVista
plotter = pv.Plotter()
plotter.add_mesh(point_cloud, color='blue', point_size=1.0, render_points_as_spheres=True)
plotter.show()



In [ ]:
import numpy as np

# Get the coordinates of the point cloud
points = point_cloud.points

# Extract y and z coordinates
yz_points = points[:, 1:3]


In [ ]:
import matplotlib.pyplot as plt

# Extract y and z coordinates
y_coords = yz_points[:, 0]
z_coords = yz_points[:, 1]

# Create a 2D scatter plot
plt.figure(figsize=(8, 6))
plt.scatter(y_coords, z_coords, c='blue', s=1)  # c is color, s is point size
plt.xlabel('Y Coordinate')
plt.ylabel('Z Coordinate')
plt.title('2D Projection of Point Cloud along YZ Plane')
plt.grid(True)
plt.show()


In [ ]:
import pyvista as pv
import numpy as np
import matplotlib.pyplot as plt

# Optional: Your point cloud object
# point_cloud = ...  # Load or create your point cloud data

# Extract coordinates
points = point_cloud.points
xy_points = points[:, 0:2]

# Extract x and y coordinates
x_coords = xy_points[:, 0]
y_coords = xy_points[:, 1]

# Create a 2D scatter plot
plt.figure(figsize=(8, 6))
plt.scatter(x_coords, y_coords, c='blue', s=1)  # c is color, s is point size
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.title('2D Projection of Point Cloud along XY Plane')
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
import meshio
import trimesh
import ray
import pyvista as pv

# Initialize Ray
ray.init(ignore_reinit_error=True)

# Load the STL file using meshio
mesh = meshio.read('/home/dave/Downloads/surface_cut.stl')

# Extract vertices and faces
vertices = mesh.points
faces = mesh.cells_dict['triangle']

# Define the bounds of the grid
x_min, x_max = vertices[:, 0].min(), vertices[:, 0].max()
y_min, y_max = vertices[:, 1].min(), vertices[:, 1].max()
z_min, z_max = vertices[:, 2].min(), vertices[:, 2].max()

# Define the resolution of the grid
resolution = 50  # Reduce for faster testing

# Create the grid
x = np.linspace(x_min, x_max, resolution)
y = np.linspace(y_min, y_max, resolution)
z = np.linspace(z_min, z_max, resolution)
xx, yy, zz = np.meshgrid(x, y, z, indexing='ij')
points = np.vstack([xx.ravel(), yy.ravel(), zz.ravel()]).T

# Convert the mesh data to a trimesh object
tri_mesh = trimesh.Trimesh(vertices=vertices, faces=faces)

# Function to check points in chunks and assign values
@ray.remote
def check_points_chunk(chunk):
    inside = tri_mesh.contains(chunk)
    return np.where(inside, 0, 1)

# Split points into chunks for parallel processing
num_chunks = 8
chunks = np.array_split(points, num_chunks)

# Run the point-in-mesh check in parallel using Ray
futures = [check_points_chunk.remote(chunk) for chunk in chunks]
values_chunks = ray.get(futures)

# Combine the results
values = np.concatenate(values_chunks)

# Create the tensor of points and their values
tensor = np.hstack((points, values.reshape(-1, 1)))

# Reshape the inside array to the grid shape
grid_values = values.reshape((resolution, resolution, resolution))

# Create a PyVista StructuredGrid from the grid values
structured_grid = pv.StructuredGrid(xx, yy, zz)
structured_grid.point_data['value'] = grid_values.ravel()

# Save the StructuredGrid to a .vts file
structured_grid.save('grid_values.vtk')

# Save the tensor to different file formats
# CSV File
np.savetxt('points_with_values.csv', tensor, delimiter=',', header='x,y,z,value', comments='')

# TXT File
np.savetxt('points_with_values.txt', tensor, fmt='%.6f', header='x y z value', comments='')

# Binary NumPy File
np.save('points_with_values.npy', tensor)

# PyVista PLY File
point_cloud = pv.PolyData(points)
point_cloud.point_data['value'] = values
point_cloud.save('points_with_values.ply')

# Optional: Visualize with PyVista
plotter = pv.Plotter()
plotter.add_mesh(point_cloud, scalars='value', cmap='coolwarm', point_size=1.0, render_points_as_spheres=True)
plotter.show()

ray.shutdown()


In [ ]:
import numpy as np
import meshio
import trimesh
import ray
import pyvista as pv

# Initialize Ray
ray.init(ignore_reinit_error=True)

# Load the STL file using meshio
mesh = meshio.read('/home/dave/Downloads/surface_cut.stl')

# Extract vertices and faces
vertices = mesh.points
faces = mesh.cells_dict['triangle']

# Convert the faces array to the format required by PyVista
faces_pv = np.hstack([np.full((faces.shape[0], 1), 3), faces]).flatten()

# Create a PyVista PolyData object from the vertices and faces
pv_mesh = pv.PolyData(vertices, faces_pv)

# Smooth the mesh
smoothed_mesh = pv_mesh.smooth(n_iter=30, relaxation_factor=0.1)

# Extract the smoothed vertices and faces
vertices = smoothed_mesh.points
faces = smoothed_mesh.faces.reshape(-1, 4)[:, 1:]

print(f"Original mesh: {mesh.points.shape[0]} vertices, {len(mesh.cells_dict['triangle'])} faces")

# Define the bounds of the grid
x_min, x_max = vertices[:, 0].min(), vertices[:, 0].max()
y_min, y_max = vertices[:, 1].min(), vertices[:, 1].max()
z_min, z_max = vertices[:, 2].min(), vertices[:, 2].max()
print(f"Bounds: x [{x_min}, {x_max}], y [{y_min}, {y_max}], z [{z_min}, {z_max}]")

# Define the resolution of the grid
resolution = 40  # Reduce for faster testing

# Create the grid
x = np.linspace(x_min, x_max, resolution)
y = np.linspace(y_min, y_max, resolution)
z = np.linspace(z_min, z_max, resolution)
xx, yy, zz = np.meshgrid(x, y, z, indexing='ij')
points = np.vstack([xx.ravel(), yy.ravel(), zz.ravel()]).T

# Convert the mesh data to a trimesh object
tri_mesh = trimesh.Trimesh(vertices=vertices, faces=faces)

# Function to check points in chunks and assign values
@ray.remote
def check_points_chunk(chunk):
    inside = tri_mesh.contains(chunk)
    return np.where(inside, 0, 1)

# Split points into chunks for parallel processing
num_chunks = 8
chunks = np.array_split(points, num_chunks)

# Run the point-in-mesh check in parallel using Ray
futures = [check_points_chunk.remote(chunk) for chunk in chunks]
values_chunks = ray.get(futures)

# Combine the results
values = np.concatenate(values_chunks)

# Create the tensor of points and their values
tensor = np.hstack((points, values.reshape(-1, 1)))

# Reshape the inside array to the grid shape
grid_values = values.reshape((resolution, resolution, resolution))

# Create a PyVista StructuredGrid from the grid values
structured_grid = pv.StructuredGrid(xx, yy, zz)
structured_grid.point_data['value'] = grid_values.ravel()

# Save the StructuredGrid to a .vts file
structured_grid.save('grid_values.vtk')

# Save the tensor to different file formats
# CSV File
np.savetxt('points_with_values.csv', tensor, delimiter=',', header='x,y,z,value', comments='')

# TXT File
np.savetxt('points_with_values.txt', tensor, fmt='%.6f', header='x y z value', comments='')

# Binary NumPy File
np.save('points_with_values.npy', tensor)

# PyVista PLY File
point_cloud = pv.PolyData(points)
point_cloud.point_data['value'] = values
point_cloud.save('points_with_values.ply')

# Optional: Visualize with PyVista
plotter = pv.Plotter()
plotter.add_mesh(point_cloud, scalars='value', cmap='coolwarm', point_size=1.0, render_points_as_spheres=True)
plotter.show()

ray.shutdown()


In [57]:
import numpy as np
import meshio
import trimesh
import ray
import pyvista as pv

# Initialize Ray
ray.init(ignore_reinit_error=True)

# Load the STL file using meshio
mesh = meshio.read('/home/dave/Downloads/surface_cut.stl')

# Extract vertices and faces
vertices = mesh.points
faces = mesh.cells_dict['triangle']

# Determine the bounding box dimensions
x_min, x_max = vertices[:, 0].min(), vertices[:, 0].max()
y_min, y_max = vertices[:, 1].min(), vertices[:, 1].max()
z_min, z_max = vertices[:, 2].min(), vertices[:, 2].max()

# Calculate the maximum dimension of the bounding box
max_dimension = max(x_max - x_min, y_max - y_min, z_max - z_min)

# Normalize vertices using the maximum dimension
vertices[:, 0] = (vertices[:, 0] - x_min) / max_dimension
vertices[:, 1] = (vertices[:, 1] - y_min) / max_dimension
vertices[:, 2] = (vertices[:, 2] - z_min) / max_dimension

# Convert the faces array to the format required by PyVista
faces_pv = np.hstack([np.full((faces.shape[0], 1), 3), faces]).flatten()

# Create a PyVista PolyData object from the vertices and faces
pv_mesh = pv.PolyData(vertices, faces_pv)

# Smooth the mesh
smoothed_mesh = pv_mesh.smooth(n_iter=30, relaxation_factor=0.1)

# Extract the smoothed vertices and faces
vertices = smoothed_mesh.points
faces = smoothed_mesh.faces.reshape(-1, 4)[:, 1:]

print(f"Original mesh: {mesh.points.shape[0]} vertices, {len(mesh.cells_dict['triangle'])} faces")

# Define the bounds of the grid
x_min, x_max = vertices[:, 0].min(), vertices[:, 0].max()
y_min, y_max = vertices[:, 1].min(), vertices[:, 1].max()
z_min, z_max = vertices[:, 2].min(), vertices[:, 2].max()
print(f"Bounds: x [{x_min}, {x_max}], y [{y_min}, {y_max}], z [{z_min}, {z_max}]")

# Define the resolution of the grid
resolution = 40  # Reduce for faster testing

# Create the grid
x = np.linspace(x_min, x_max, resolution)
y = np.linspace(y_min, y_max, resolution)
z = np.linspace(z_min, z_max, resolution)
xx, yy, zz = np.meshgrid(x, y, z, indexing='ij')
points = np.vstack([xx.ravel(), yy.ravel(), zz.ravel()]).T

# Convert the mesh data to a trimesh object
tri_mesh = trimesh.Trimesh(vertices=vertices, faces=faces)

# Function to check points in chunks and assign values
@ray.remote
def check_points_chunk(chunk):
    print(chunk)
    inside = tri_mesh.contains(chunk)
    return np.where(inside, 0, 1)

# Split points into chunks for parallel processing
num_chunks = 8
chunks = np.array_split(points, num_chunks)

# Run the point-in-mesh check in parallel using Ray
futures = [check_points_chunk.remote(chunk) for chunk in chunks]
values_chunks = ray.get(futures)

# Combine the results
values = np.concatenate(values_chunks)

# Create the tensor of points and their values
tensor = np.hstack((points, values.reshape(-1, 1)))

# Reshape the inside array to the grid shape
grid_values = values.reshape((resolution, resolution, resolution))

# Create a PyVista StructuredGrid from the grid values
structured_grid = pv.StructuredGrid(xx, yy, zz)
structured_grid.point_data['value'] = grid_values.ravel()

# Save the StructuredGrid to a .vts file
structured_grid.save('grid_values.vtk')

# Save the tensor to different file formats
# CSV File
np.savetxt('points_with_values.csv', tensor, delimiter=',', header='x,y,z,value', comments='')

# TXT File
np.savetxt('points_with_values.txt', tensor, fmt='%.6f', header='x y z value', comments='')

# Binary NumPy File
np.save('points_with_values.npy', tensor)

# PyVista PLY File
point_cloud = pv.PolyData(points)
point_cloud.point_data['value'] = values
point_cloud.save('points_with_values.ply')

# Optional: Visualize with PyVista
plotter = pv.Plotter()
plotter.add_mesh(point_cloud, scalars='value', cmap='coolwarm', point_size=1.0, render_points_as_spheres=True)
plotter.show()

ray.shutdown()


2024-08-04 23:06:14,992	INFO worker.py:1788 -- Started a local Ray instance.


Original mesh: 1099 vertices, 2091 faces
Bounds: x [0.009741447865962982, 0.4689101576805115], y [0.0, 1.0], z [0.006530439015477896, 0.43905526399612427]
(check_points_chunk pid=53846) [[0.24521258 0.         0.00653044]
(check_points_chunk pid=53846)  [0.24521258 0.         0.01762082]
(check_points_chunk pid=53846)  [0.24521258 0.         0.0287112 ]
(check_points_chunk pid=53846)  ...
(check_points_chunk pid=53846)  [0.29230681 1.         0.4168745 ]
(check_points_chunk pid=53846)  [0.29230681 1.         0.42796488]
(check_points_chunk pid=53846)  [0.29230681 1.         0.43905526]]
(check_points_chunk pid=53844) [[0.00974145 0.         0.00653044]
(check_points_chunk pid=53844)  [0.00974145 0.         0.01762082]
(check_points_chunk pid=53844)  [0.05683567 1.         0.42796488]
(check_points_chunk pid=53844)  [0.05683567 1.         0.43905526]]
(check_points_chunk pid=53845) [[0.1863448  0.         0.00653044]
(check_points_chunk pid=53845)  [0.1863448  0.         0.01762082]
(ch

KeyboardInterrupt: 

In [ ]:
import petsc4py
petsc4py.init()
from petsc4py import PETSc
from stl import mesh
import numpy as np
import pyvista as pv

def create_dmstag_3d():
    # Create a DMStag object
    dmstag = PETSc.DMStag().create(dim=3)

    # Set grid sizes for each dimension
    nx, ny, nz = 10, 10, 10
    dmstag.setGlobalSizes([nx, ny, nz])

    # Set boundary types for each dimension
    boundary_types = [PETSc.DM.BoundaryType.NONE, PETSc.DM.BoundaryType.NONE, PETSc.DM.BoundaryType.NONE]
    dmstag.setBoundaryTypes(boundary_types)

    # Set number of dof (degrees of freedom) per cell (corner, edge, face, and element)
    dof_per_element = [0, 0, 1, 0]
    dmstag.setDof(dof_per_element)

    # Set up the DMStag
    dmstag.setFromOptions()

    # Finalize the DM setup
    dmstag.setUp()

    # Set uniform coordinates explicitly
    dmstag.setUniformCoordinatesExplicit(0, 1, 0, 1, 0, 1)
    return dmstag

def is_point_in_mesh(point, mesh):
    """Check if a point is inside a mesh using ray-casting."""
    polydata = pv.PolyData(mesh.vectors.reshape(-1, 3))
    return polydata.select_enclosed_points(pv.PolyData([point]), tolerance=0.0).point_data['SelectedPoints'][0] == 1

def main():
    # Load the STL file
    stl_mesh = mesh.Mesh.from_file('/home/dave/Downloads/surface_cut.stl')
    
    # Create the 3D DMStag object
    dmstag = create_dmstag_3d()

    # Get the global sizes
    nx, ny, nz = dmstag.getGlobalSizes()

    # Create a Vec to store the data
    localX = dmstag.createLocalVec()
    globalX = dmstag.createGlobalVec()

    # Get the local grid coordinates
    coords_local = dmstag.getCoordinatesLocal().array

    # Reshape coordinates to match the grid points
    #num_points = (nx + 1) * (ny + 1) * (nz + 1)

    coords_local = coords_local.reshape(3993, 3)


    # Create a numpy array to hold the values
    values = np.ones(3993, dtype=int)

    # Check each grid point and set the values in the localX Vec
    for idx, coord in enumerate(coords_local):
        if is_point_in_mesh(coord, stl_mesh):
            values[idx] = 0  # Inside the carotid surface

    # Set the local vector with the computed values
    localX.setArray(values)

    # Scatter local vector to global vector
    dmstag.localToGlobal(localX, globalX)
    # Print DMStag information
    dmstag.view()

    # Optionally, save the global vector for further processing or visualization
    viewer = PETSc.Viewer().createASCII('output.txt', 'w')
    viewer.view(globalX)


if __name__ == "__main__":
    main()


In [ ]:
import petsc4py
petsc4py.init()
from petsc4py import PETSc
from stl import mesh
import numpy as np
import pyvista as pv

def create_dmstag_3d():
    # Create a DMStag object
    dmstag = PETSc.DMStag().create(dim=3)

    # Set grid sizes for each dimension
    nx, ny, nz = 10, 10, 10
    dmstag.setGlobalSizes([nx, ny, nz])

    # Set boundary types for each dimension
    boundary_types = [PETSc.DM.BoundaryType.NONE, PETSc.DM.BoundaryType.NONE, PETSc.DM.BoundaryType.NONE]
    dmstag.setBoundaryTypes(boundary_types)

    # Set number of dof (degrees of freedom) per cell (corner, edge, face, and element)
    dof_per_element = [0, 0, 1, 0]
    dmstag.setDof(dof_per_element)

    # Set up the DMStag
    dmstag.setFromOptions()

    # Finalize the DM setup
    dmstag.setUp()

    # Set uniform coordinates explicitly
    dmstag.setUniformCoordinatesExplicit(0, 1, 0, 1, 0, 1)
    return dmstag

def is_point_in_mesh(point, mesh):
    """Check if a point is inside a mesh using ray-casting."""

    
    polydata = pv.PolyData(mesh.vectors.reshape(-1, 3))
    return polydata.select_enclosed_points(pv.PolyData([point]), tolerance=0.0).point_data['SelectedPoints'][0] == 1

def save_vector_vtr(dmstag, vec, filename):
    """Save the vector in VTR format."""
    # Create a PETSc viewer for VTK output
    viewer = PETSc.Viewer().createVTK(filename, 'w')
    
    # View the vector using the VTK viewer
    vec.view(viewer)
    
    # Clean up
    viewer.destroy()

def main():
    # Load the STL file
    stl_mesh = mesh.Mesh.from_file('/home/dave/Downloads/surface_cut.stl')

    """Normalize the STL mesh coordinates to fit within [0, 1] range."""
    # Extract vertices from the STL mesh
    vertices = stl_mesh.vectors.reshape(-1, 3)
    
    # Compute the min and max coordinates
    min_coords = vertices.min(axis=0)
    max_coords = vertices.max(axis=0)
    
    # Compute the range (size) of each dimension
    ranges = max_coords - min_coords
    
    # Determine the maximum range for normalization
    max_range = ranges.max()
    
    # Normalize the vertices
    normalized_vertices = (vertices - min_coords) / max_range
    
    # Update the mesh with normalized vertices
    # STL mesh expects the vertices to be reshaped in the original format
    stl_mesh.vectors = normalized_vertices.reshape(-1, 3, 3)
    
    # Print normalized bounding box
    normalized_min_coords = normalized_vertices.min(axis=0)
    normalized_max_coords = normalized_vertices.max(axis=0)
    #print(f"Normalized minimum coordinates: {normalized_min_coords}")
    #print(f"Normalized maximum coordinates: {normalized_max_coords}")

    # Create the 3D DMStag object
    dmstag = create_dmstag_3d()

    # Get the global sizes
    nx, ny, nz = dmstag.getGlobalSizes()

    # Create a Vec to store the data
    localX = dmstag.createLocalVec()
    globalX = dmstag.createGlobalVec()

    # Get the local grid coordinates
    coords_local = dmstag.getCoordinatesLocal().array

    # Ensure correct reshaping based on grid size
    coords_local = coords_local.reshape((3993, 3))

    # Create a numpy array to hold the values
    values = np.ones(3993, dtype=int)

    # Check each grid point and set the values in the localX Vec
    for idx, coord in enumerate(coords_local):
        if is_point_in_mesh(coord, stl_mesh):
            values[idx] = 0  # Inside the carotid surface

    # Set the local vector with the computed values
    localX.setArray(values)

    # Scatter local vector to global vector
    dmstag.localToGlobal(localX, globalX)

    # Print the global vector to the terminal
    print("Global vector contents:")
    #globalX.view()
    # Print DMStag information
    dmstag.view()

    # Save the global vector in VTR format
    save_vector_vtr(dmstag, globalX, 'output.vtr')

if __name__ == "__main__":
    main()


In [56]:
import petsc4py
petsc4py.init()
from petsc4py import PETSc
import numpy as np
import pyvista as pv
import meshio
import trimesh



def create_dmstag_3d():
    # Create a DMStag object
    dmstag = PETSc.DMStag().create(dim=3)

    # Set grid sizes for each dimension
    nx, ny, nz = 10, 10, 10
    dmstag.setGlobalSizes([nx, ny, nz])

    # Set boundary types for each dimension
    boundary_types = [PETSc.DM.BoundaryType.NONE, PETSc.DM.BoundaryType.NONE, PETSc.DM.BoundaryType.NONE]
    dmstag.setBoundaryTypes(boundary_types)

    # Set number of dof (degrees of freedom) per cell (corner, edge, face, and element)
    dof_per_element = [0, 0, 1, 0]
    dmstag.setDof(dof_per_element)

    # Set up the DMStag
    dmstag.setFromOptions()

    # Finalize the DM setup
    dmstag.setUp()

    # Set uniform coordinates explicitly
    dmstag.setUniformCoordinatesExplicit(0, 1, 0, 1, 0, 1)
    return dmstag

def is_point_in_mesh(point, mesh):
    """Check if a point is inside a mesh using ray-casting."""

    
    polydata = pv.PolyData(mesh.vectors.reshape(-1, 3))
    return polydata.select_enclosed_points(pv.PolyData([point]), tolerance=0.0).point_data['SelectedPoints'][0] == 1

def save_vector_vtr(dmstag, vec, filename):
    """Save the vector in VTR format."""
    # Create a PETSc viewer for VTK output
    viewer = PETSc.Viewer().createVTK(filename, 'w')
    
    # View the vector using the VTK viewer
    vec.view(viewer)
    
    # Clean up
    viewer.destroy()

def main():

    # Load the STL file using meshio
    stl_mesh = meshio.read('/home/dave/Downloads/surface_cut.stl')

    # Extract vertices and faces
    vertices = mesh.points
    faces = mesh.cells_dict['triangle']

    # Determine the bounding box dimensions
    x_min, x_max = vertices[:, 0].min(), vertices[:, 0].max()
    y_min, y_max = vertices[:, 1].min(), vertices[:, 1].max()
    z_min, z_max = vertices[:, 2].min(), vertices[:, 2].max()

    # Calculate the maximum dimension of the bounding box
    max_dimension = max(x_max - x_min, y_max - y_min, z_max - z_min)

    # Normalize vertices using the maximum dimension
    vertices[:, 0] = (vertices[:, 0] - x_min) / max_dimension
    vertices[:, 1] = (vertices[:, 1] - y_min) / max_dimension
    vertices[:, 2] = (vertices[:, 2] - z_min) / max_dimension

    # Convert the faces array to the format required by PyVista
    faces_pv = np.hstack([np.full((faces.shape[0], 1), 3), faces]).flatten()

    # Create a PyVista PolyData object from the vertices and faces
    pv_mesh = pv.PolyData(vertices, faces_pv)

    # Smooth the mesh
    smoothed_mesh = pv_mesh.smooth(n_iter=30, relaxation_factor=0.1)

    # Extract the smoothed vertices and faces
    vertices = smoothed_mesh.points
    faces = smoothed_mesh.faces.reshape(-1, 4)[:, 1:]

    print(f"Original mesh: {mesh.points.shape[0]} vertices, {len(mesh.cells_dict['triangle'])} faces")

    # Define the bounds of the grid
    x_min, x_max = vertices[:, 0].min(), vertices[:, 0].max()
    y_min, y_max = vertices[:, 1].min(), vertices[:, 1].max()
    z_min, z_max = vertices[:, 2].min(), vertices[:, 2].max()
    print(f"Bounds: x [{x_min}, {x_max}], y [{y_min}, {y_max}], z [{z_min}, {z_max}]")

    # Convert the mesh data to a trimesh object
    stl_mesh = trimesh.Trimesh(vertices=vertices, faces=faces)

    print("ciao")

    # Create the 3D DMStag object
    dmstag = create_dmstag_3d()

    # Get the global sizes
    nx, ny, nz = dmstag.getGlobalSizes()

    # Create a Vec to store the data
    localX = dmstag.createLocalVec()
    globalX = dmstag.createGlobalVec()

    # Get the local grid coordinates
    coords_local = dmstag.getCoordinatesLocal().array

    # Ensure correct reshaping based on grid size
    coords_local = coords_local.reshape((3993, 3))

    # Create a numpy array to hold the values
    values = np.ones(3993, dtype=int)

    

    # Check each grid point and set the values in the localX Vec
    for idx, coord in enumerate(coords_local):
        if is_point_in_mesh(coord, stl_mesh):
            values[idx] = 0  # Inside the carotid surface

    # Set the local vector with the computed values
    localX.setArray(values)

    # Scatter local vector to global vector
    dmstag.localToGlobal(localX, globalX)

    # Print the global vector to the terminal
    print("Global vector contents:")
    #globalX.view()
    # Print DMStag information
    dmstag.view()

    # Save the global vector in VTR format
    save_vector_vtr(dmstag, globalX, 'output.vtr')

if __name__ == "__main__":
    main()


AttributeError: module 'stl.mesh' has no attribute 'points'